In [1]:
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import time
import numpy as np
import matplotlib.pyplot as plt
import math
from deep_audio import Directory, JSON

In [2]:
method_algo = 'lpcc'
model_algo = 'rnn'
n_rate = 24000

DATASET_PATH = f'processed/{method_algo}/{method_algo}_{n_rate}.json'

inputs, targets, mapping = Directory.load_json_data(DATASET_PATH, inputs_fieldname=method_algo)

In [3]:
def create_dense_model():
    # build the network architecture
    model = keras.Sequential([
        # 1st hidden layer
        keras.layers.SimpleRNN(512, input_shape=[inputs.shape[1], inputs.shape[2]]),

        keras.layers.Dropout(0.3),

        # output layer
        keras.layers.Dense(len(mapping), activation='softmax'),
    ])

    return model

In [4]:
# SPLIT DOS DADOS
random_state = 42
# for random_state in [5438, 53, 14]:
#     for _ in range(4):
# split data into train and test set
inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs,
                                                                          targets,
                                                                          test_size=0.2,
                                                                          stratify=targets,
                                                                          random_state=random_state)

inputs_train, inputs_valid, targets_train, targets_valid = train_test_split(inputs_train,
                                                                            targets_train,
                                                                            test_size=0.2,
                                                                            stratify=targets_train,
                                                                            random_state=random_state)


In [5]:
# SALVA UMA CÓPIA DOS DADOS DE TESTE

# data = {
#     "mapping": mapping,
#     "labels": targets_test,
#     "mfcc": inputs_test,
# }

# JSON.create_json_file(f'datatest/deep/datatest_{random_state}_{inputs.shape[0]}.json', data)

In [6]:
# CRIA O MODELO
model = create_dense_model()

# COMPILA A REDE
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
# SALVA A ESTRUTURA DO MODELO

timestamp = int(time.time())

Directory.create_directory(f'models/{model_algo}/{method_algo}/{timestamp}')

JSON.create_json_file(f'models/{model_algo}/{method_algo}/{timestamp}/model_structure.json', model.to_json())

model_save_filename = f'models/{model_algo}/{method_algo}/{timestamp}/model_weight.h5'

# DECIDE QUANDO PARAR
earlystopping_cb = keras.callbacks.EarlyStopping(patience=300, restore_best_weights=True)

# SALVA OS PESOS
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
    model_save_filename, monitor="val_accuracy", save_best_only=True
)

In [ ]:
# TREINA O MODELO
history = model.fit(inputs_train, targets_train,
                    validation_data=(inputs_valid, targets_valid),
                    epochs=10000,
                    batch_size=128,
                    callbacks=[earlystopping_cb, mdlcheckpoint_cb])

Epoch 1/10000
89/89 [==============================] - 13s 134ms/step - loss: 5.3731 - accuracy: 0.0101 - val_loss: 4.7908 - val_accuracy: 0.0110
Epoch 2/10000
89/89 [==============================] - 9s 101ms/step - loss: 5.1210 - accuracy: 0.0074 - val_loss: 4.7305 - val_accuracy: 0.0113
Epoch 3/10000
89/89 [==============================] - 10s 110ms/step - loss: 5.0582 - accuracy: 0.0115 - val_loss: 4.7050 - val_accuracy: 0.0170
Epoch 4/10000
89/89 [==============================] - 10s 108ms/step - loss: 5.0214 - accuracy: 0.0119 - val_loss: 4.7090 - val_accuracy: 0.0163
Epoch 5/10000
89/89 [==============================] - 8s 92ms/step - loss: 5.0406 - accuracy: 0.0117 - val_loss: 4.7048 - val_accuracy: 0.0141
Epoch 6/10000
89/89 [==============================] - 8s 91ms/step - loss: 5.0408 - accuracy: 0.0099 - val_loss: 4.6925 - val_accuracy: 0.0138
Epoch 7/10000
89/89 [==============================] - 8s 91ms/step - loss: 5.0227 - accuracy: 0.0108 - val_loss: 4.6980 - val_ac

89/89 [==============================] - 8s 92ms/step - loss: 4.8829 - accuracy: 0.0125 - val_loss: 4.6628 - val_accuracy: 0.0156
Epoch 58/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.9096 - accuracy: 0.0100 - val_loss: 4.6602 - val_accuracy: 0.0191
Epoch 59/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.8747 - accuracy: 0.0121 - val_loss: 4.6587 - val_accuracy: 0.0159
Epoch 60/10000
89/89 [==============================] - 8s 91ms/step - loss: 4.8752 - accuracy: 0.0125 - val_loss: 4.6672 - val_accuracy: 0.0145
Epoch 61/10000
89/89 [==============================] - 8s 91ms/step - loss: 4.8955 - accuracy: 0.0102 - val_loss: 4.6595 - val_accuracy: 0.0170
Epoch 62/10000
89/89 [==============================] - 9s 107ms/step - loss: 4.8844 - accuracy: 0.0132 - val_loss: 4.6631 - val_accuracy: 0.0195
Epoch 63/10000
89/89 [==============================] - 9s 102ms/step - loss: 4.9059 - accuracy: 0.0115 - val_loss: 4.6589 - val_accuracy: 0.014

89/89 [==============================] - 9s 100ms/step - loss: 4.7524 - accuracy: 0.0142 - val_loss: 4.6090 - val_accuracy: 0.0191
Epoch 114/10000
89/89 [==============================] - 9s 103ms/step - loss: 4.7333 - accuracy: 0.0155 - val_loss: 4.6050 - val_accuracy: 0.0202
Epoch 115/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.7467 - accuracy: 0.0151 - val_loss: 4.6101 - val_accuracy: 0.0205
Epoch 116/10000
89/89 [==============================] - 9s 100ms/step - loss: 4.7472 - accuracy: 0.0167 - val_loss: 4.6041 - val_accuracy: 0.0198
Epoch 117/10000
89/89 [==============================] - 9s 100ms/step - loss: 4.7533 - accuracy: 0.0136 - val_loss: 4.6030 - val_accuracy: 0.0202
Epoch 118/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.7348 - accuracy: 0.0158 - val_loss: 4.6072 - val_accuracy: 0.0198
Epoch 119/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.7441 - accuracy: 0.0159 - val_loss: 4.6015 - val_accurac

Epoch 169/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.5911 - accuracy: 0.0204 - val_loss: 4.5104 - val_accuracy: 0.0241
Epoch 170/10000
89/89 [==============================] - 9s 103ms/step - loss: 4.5891 - accuracy: 0.0191 - val_loss: 4.5089 - val_accuracy: 0.0223
Epoch 171/10000
89/89 [==============================] - 8s 91ms/step - loss: 4.5901 - accuracy: 0.0205 - val_loss: 4.5069 - val_accuracy: 0.0262
Epoch 172/10000
89/89 [==============================] - 9s 101ms/step - loss: 4.6082 - accuracy: 0.0197 - val_loss: 4.5149 - val_accuracy: 0.0255
Epoch 173/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.5998 - accuracy: 0.0242 - val_loss: 4.5032 - val_accuracy: 0.0237
Epoch 174/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.5923 - accuracy: 0.0189 - val_loss: 4.5066 - val_accuracy: 0.0269
Epoch 175/10000
89/89 [==============================] - 9s 97ms/step - loss: 4.5977 - accuracy: 0.0190 - val_loss: 4.5026

Epoch 225/10000
89/89 [==============================] - 9s 96ms/step - loss: 4.4602 - accuracy: 0.0250 - val_loss: 4.4224 - val_accuracy: 0.0258
Epoch 226/10000
89/89 [==============================] - 9s 102ms/step - loss: 4.4545 - accuracy: 0.0241 - val_loss: 4.4007 - val_accuracy: 0.0287
Epoch 227/10000
89/89 [==============================] - 9s 101ms/step - loss: 4.4546 - accuracy: 0.0262 - val_loss: 4.4003 - val_accuracy: 0.0248
Epoch 228/10000
89/89 [==============================] - 9s 97ms/step - loss: 4.4349 - accuracy: 0.0248 - val_loss: 4.4005 - val_accuracy: 0.0226
Epoch 229/10000
89/89 [==============================] - 11s 118ms/step - loss: 4.4476 - accuracy: 0.0256 - val_loss: 4.3981 - val_accuracy: 0.0287
Epoch 230/10000
89/89 [==============================] - 10s 109ms/step - loss: 4.4301 - accuracy: 0.0244 - val_loss: 4.4064 - val_accuracy: 0.0248
Epoch 231/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.4550 - accuracy: 0.0238 - val_loss: 4.

...

89/89 [==============================] - 8s 95ms/step - loss: 4.0880 - accuracy: 0.0491 - val_loss: 4.1337 - val_accuracy: 0.0318
Epoch 730/10000
89/89 [==============================] - 10s 113ms/step - loss: 4.0788 - accuracy: 0.0531 - val_loss: 4.1578 - val_accuracy: 0.0350
Epoch 731/10000
89/89 [==============================] - 9s 107ms/step - loss: 4.1088 - accuracy: 0.0443 - val_loss: 4.1465 - val_accuracy: 0.0340
Epoch 732/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.0902 - accuracy: 0.0455 - val_loss: 4.1324 - val_accuracy: 0.0340
Epoch 733/10000
89/89 [==============================] - 9s 103ms/step - loss: 4.1020 - accuracy: 0.0419 - val_loss: 4.1274 - val_accuracy: 0.0357
Epoch 734/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.0876 - accuracy: 0.0424 - val_loss: 4.1352 - val_accuracy: 0.0322
Epoch 735/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.0855 - accuracy: 0.0440 - val_loss: 4.1329 - val_accurac

89/89 [==============================] - 8s 92ms/step - loss: 4.0699 - accuracy: 0.0481 - val_loss: 4.1226 - val_accuracy: 0.0375
Epoch 786/10000
89/89 [==============================] - 12s 133ms/step - loss: 4.0970 - accuracy: 0.0453 - val_loss: 4.1247 - val_accuracy: 0.0333
Epoch 787/10000
89/89 [==============================] - 8s 94ms/step - loss: 4.0919 - accuracy: 0.0422 - val_loss: 4.1233 - val_accuracy: 0.0375
Epoch 788/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.0611 - accuracy: 0.0496 - val_loss: 4.1242 - val_accuracy: 0.0343
Epoch 789/10000
89/89 [==============================] - 10s 112ms/step - loss: 4.0779 - accuracy: 0.0499 - val_loss: 4.1320 - val_accuracy: 0.0304
Epoch 790/10000
89/89 [==============================] - 10s 107ms/step - loss: 4.0665 - accuracy: 0.0483 - val_loss: 4.1237 - val_accuracy: 0.0368
Epoch 791/10000
89/89 [==============================] - 9s 105ms/step - loss: 4.0680 - accuracy: 0.0483 - val_loss: 4.1354 - val_accu

89/89 [==============================] - 10s 110ms/step - loss: 4.1003 - accuracy: 0.0480 - val_loss: 4.1083 - val_accuracy: 0.0364
Epoch 842/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.0459 - accuracy: 0.0537 - val_loss: 4.1252 - val_accuracy: 0.0364
Epoch 843/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.0702 - accuracy: 0.0505 - val_loss: 4.1331 - val_accuracy: 0.0361
Epoch 844/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.0960 - accuracy: 0.0495 - val_loss: 4.1073 - val_accuracy: 0.0357
Epoch 845/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.0684 - accuracy: 0.0501 - val_loss: 4.1104 - val_accuracy: 0.0347
Epoch 846/10000
89/89 [==============================] - 10s 111ms/step - loss: 4.0640 - accuracy: 0.0468 - val_loss: 4.1228 - val_accuracy: 0.0400
Epoch 847/10000
89/89 [==============================] - 9s 99ms/step - loss: 4.0695 - accuracy: 0.0468 - val_loss: 4.1080 - val_accurac

89/89 [==============================] - 9s 104ms/step - loss: 4.0595 - accuracy: 0.0499 - val_loss: 4.1227 - val_accuracy: 0.0340
Epoch 898/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.0553 - accuracy: 0.0449 - val_loss: 4.1081 - val_accuracy: 0.0336
Epoch 899/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.0788 - accuracy: 0.0442 - val_loss: 4.1129 - val_accuracy: 0.0368
Epoch 900/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.0723 - accuracy: 0.0471 - val_loss: 4.1532 - val_accuracy: 0.0343
Epoch 901/10000
89/89 [==============================] - 8s 91ms/step - loss: 4.0644 - accuracy: 0.0474 - val_loss: 4.1138 - val_accuracy: 0.0354
Epoch 902/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.0565 - accuracy: 0.0455 - val_loss: 4.1172 - val_accuracy: 0.0340
Epoch 903/10000
89/89 [==============================] - 8s 91ms/step - loss: 4.0542 - accuracy: 0.0535 - val_loss: 4.1060 - val_accuracy: 

89/89 [==============================] - 9s 103ms/step - loss: 4.0515 - accuracy: 0.0476 - val_loss: 4.1643 - val_accuracy: 0.0336
Epoch 954/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.0715 - accuracy: 0.0497 - val_loss: 4.0978 - val_accuracy: 0.0378
Epoch 955/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.0480 - accuracy: 0.0508 - val_loss: 4.0930 - val_accuracy: 0.0364
Epoch 956/10000
89/89 [==============================] - 10s 114ms/step - loss: 4.0422 - accuracy: 0.0475 - val_loss: 4.0903 - val_accuracy: 0.0364
Epoch 957/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.0594 - accuracy: 0.0492 - val_loss: 4.0917 - val_accuracy: 0.0386
Epoch 958/10000
89/89 [==============================] - 8s 92ms/step - loss: 4.0355 - accuracy: 0.0515 - val_loss: 4.1264 - val_accuracy: 0.0347
Epoch 959/10000
89/89 [==============================] - 8s 93ms/step - loss: 4.0862 - accuracy: 0.0487 - val_loss: 4.0831 - val_accuracy

In [ ]:
# GERA O GRAFICO DE ACURÁCIA
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/{model_algo}/{method_algo}/{timestamp}/graph_accuracy.png')
plt.close()

# GERA O GRÁFICO DE PERCA
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/{model_algo}/{method_algo}/{timestamp}/graph_loss.png')
plt.close()

In [ ]:
# PEGA A MAIOR E ACURÁCIA
higher_accuracy = model.evaluate(inputs_test, targets_test, batch_size=128)

higher_accuracy = str(int(higher_accuracy[1] * 10000)).zfill(4)

In [ ]:
# RENOMEIA A PASTA
Directory.rename_directory(f'models/{model_algo}/{method_algo}/{timestamp}',
                           f'models/{model_algo}/{method_algo}/acc{higher_accuracy}_seed{random_state}_epochs{len(history.history["accuracy"])}_time{timestamp}')